# Pathfinding Maze Solver
### How to run the program

Every cell of code withint the program can be run independently without having to rely on dependencies from eachother this means that it does not matter what order you run the cells they will always work. To run the program for a custom maze for each program you will need to write *algorithm("maze.txt")*. the mazes are located in the **mazes** folder, and the solutions are found in the **algorithm_path_solutions** folder.

## Q1.1 Description of how the maze solver can be seen as a search problem

The maze solver can be seen as a search problem because it is a pathfinding question with a start and goal, where a path that leads to the destination must be found and can be found, depending on the algorithm, in a certain amount of time. the # and the - are elements that determine what is a wall and what is the path. This means that a search algorithm will be able to tell what each node represents being a path or a wall. Also it can be seen as a search problem because there are multiple paths that lead to the solution however one path is better then the rest, and so different algorithms can find different paths according to different time and space complexities.

## Q1.2 Maze Solver using Depth-First Search
### Solving the maze using Depth-First Search
In depth-first search you start at the root node and as far down each branch to different nodes before backtracking. One problem with this algorithm is that there is a possibility for looping within nodes where the search algorithm will be stuck between a certain number of nodes because theres is a cycle within the tree; this means that the algorithm does not have completeness. the time complexity for the algortihm is high being *branches^maxDepth* this is because in the worst case the algorithm visits all states multiple times. the space complexity for the algorithm is low being the *maxDepth* as only one path is being remembered at a time. The algorithm also is not guaranteed to find the shortest solution, as if there are multiple solutions it will find the one that is in the closest to the inital branch.

In [1]:
# time required to calculate the total time
import time

def depth_first(m):
    '''
    This function takes bundles everything together so that when this
    is called everything is ran at the correct time.
    
        Parameters:
            m: The file of the maze that is going to be used in the algorithm
    '''
    start_j_counter = 0
    end_i_counter = 0
    end_j_counter = 0
    # Directions in 'ESNW'
    directions = [(0,1), (1,0), (-1,0), (0,-1)]

    # Opens the file and formats contents into list of lists
    with open("mazes/"+m, 'r') as file:
        maze = [list(line.replace(' ','').strip()) for line in file]
        
    # Find start and end i and j values
    start_i = 0

    for i in maze[0]:
        if i == '-':
            start_j = start_j_counter
            break
        else:
            start_j_counter += 1
            continue

    for i in maze:
        end_i_counter += 1
    end_i = end_i_counter - 1

    for i in maze[-1]:
        if i == '-':
            end_j = end_j_counter
            break
        else:
            end_j_counter += 1
            continue
            
    def dfs():
        '''
        This function will run the depth first search algorithm, where
        the cells explored, time taken to run and the path taken will
        be printed. It will also either return the path or none if there
        is no possible solution.
        '''
        start_time = time.time()
        # A set has been used because it has a search complexity of O(1)
        cells_explored = set()
        path_stack = [((start_i, start_j), [])]
        while path_stack:
            cell, path = path_stack.pop()
            if cell == (end_i, end_j):
                total_time = time.time() - start_time
                print("Total running time: " + str(total_time))
                print("Number of nodes explored: " + str(len(cells_explored)))
                print("Number of steps in the path: " + str(len(path)))
                print("check file to find the path taken for",m,"\n")
                f = open("dfs_path_solutions/dfs_"+m, "w")
                for i in path:
                    f.write(str(i))
                f.close()
                return path + [cell]
            # Checks to see if the cell has already been explored
            if cell not in cells_explored:
                # Add the current cell to the cells explored array
                cells_explored.add(cell)
                for direction in directions:
                    next_cell = (cell[0] + direction[0], cell[1] + direction[1])
                    # Check to see if each direction has a viable path
                    if maze[next_cell[0]][next_cell[1]] != "#" and next_cell not in cells_explored:
                        # Adds the next cell to the stack to check if it has a path
                        path_stack.append((next_cell, path + [cell]))
        return None
    dfs()
    
depth_first("maze-Easy.txt")
depth_first("maze-Medium.txt")
depth_first("maze-Large.txt")
depth_first("maze-VLarge.txt")

Total running time: 0.00013184547424316406
Number of nodes explored: 79
Number of steps in the path: 40
check file to find the path taken for maze-Easy.txt 

Total running time: 0.020229101181030273
Number of nodes explored: 7055
Number of steps in the path: 768
check file to find the path taken for maze-Medium.txt 

Total running time: 0.05293989181518555
Number of nodes explored: 17470
Number of steps in the path: 1141
check file to find the path taken for maze-Large.txt 

Total running time: 7.95281982421875
Number of nodes explored: 626490
Number of steps in the path: 6118
check file to find the path taken for maze-VLarge.txt 



## Q1.3 Improved Maze Solver
### A better algorithm to tackle this problem would be A Star search
A star is an algorithm that finds the shortest path from a start to an end goal. It is done by calculating a total cost by adding a heuristic  and actual cost to reach the end cell. The heuristic cost is created by calculating the absolute distance of the cell from the goal both horizontally and vertically, whereas the actual cost is calculated by adding one to the previous cells actual cost and is the distance from the starting position. We then trace back through the lowest scores to give us the shortest distance to the goal cell.

This algorithm is considered more efficient then DFS because there is always completeness and optimality meaning that A star will always find a solution and is always guaranteed to find the shortest, this is much better then DFS as DFS has neither. The time complexity of A star is *branches^depth*. In its worst case it is the same as DFS, but under most circumstances it will be faster. However the space complexity of A star is *branches^depth* which is worse then DFS.

In [2]:
# time required to calculate the total time
import time
# heapq used for fast access to lists
import heapq
# defaultdict is used to fill dictionary with infinite values
from collections import defaultdict

def a_star(m):
    '''
    This function takes bundles everything together so that when this
    is called everything is ran at the correct time.
    
        Parameters:
            m: The file of the maze that is going to be used in the algorithm
    '''
    start_j_counter = 0
    end_i_counter = 0
    end_j_counter = 0
    # Directions in 'ESNW'
    directions = [(0,1), (1,0), (-1,0), (0,-1)]

    # Opens the file and formats contents into list of lists
    with open("mazes/"+m, 'r') as file:
        maze = [list(line.replace(' ','').strip()) for line in file]

    # Find start and end i and j values
    start_i = 0

    for i in maze[0]:
        if i == '-':
            start_j = start_j_counter
            break
        else:
            start_j_counter += 1
            continue

    for i in maze:
        end_i_counter += 1
    end_i = end_i_counter-1

    for i in maze[-1]:
        if i == '-':
            end_j = end_j_counter
            break
        else:
            end_j_counter += 1
            continue
    
    # Puts start and end co-ordinates into tuple form
    start = (start_i, start_j)
    end = (end_i, end_j)
    
    def h_cost(curr):
        '''
        This function takes in the current cells value and returns
        the manhattan distance between that cell and the end cell.
        
            Parameters:
                curr: (tuple of two indices)
                
            Returns:
                Manhattan distance from current cell to end cell
        '''
        return (abs(end[0]-curr[0])+abs(end[1]-curr[1]))
    
    def astar():
        '''
        This function runs the A Star algorithm on a maze to find the
        shortest distance in a fast period of time. It will either return
        None if there is no solution or the path if there is a solution.
        '''
        start_time = time.time()
        open_cells = []
        # heapq is very efficient with its time complexity hence it is used here.
        heapq.heappush(open_cells, (h_cost(start), start))
        # A set has been used because it has a search complexity of O(1)
        closed_cells = set()
        # G values are infinity so that when new cells are found, their 
        # respective values are smaller
        g_vals = defaultdict(lambda: float('inf'))
        g_vals[start] = 0
        # Stores the parent node of each node visited
        parents = {}
        while open_cells:
            # Take the smallest amount from the heap and return's that cell
            current_cell = heapq.heappop(open_cells)[1]
            if current_cell == end:
                total_time = time.time() - start_time
                path_stack = []
                while current_cell != start:
                    path_stack.append(current_cell)
                    current_cell = parents[current_cell]
                path_stack.append(current_cell)
                reverse_path = path_stack[::-1]
                print("Total running time: " + str(total_time))
                print("Number of nodes explored: " + str(len(closed_cells)))
                print("Number of steps in the path: " + str(len(path_stack)))
                print("check file to find the path taken for",m,"\n")
                f = open("a_star_path_solutions/a_star_"+m, "w")
                for i in reverse_path:
                    f.write(str(i))
                f.close()
                return reverse_path
            closed_cells.add(current_cell)
            for direction in directions:
                next_cell = (current_cell[0] + direction[0], current_cell[1] + direction[1])
                # Check to see if each direction has a viable path
                if next_cell in closed_cells or maze[next_cell[0]][next_cell[1]] == "#":
                    continue
                # Increments the g value of the current cell
                g_val = g_vals[current_cell] + 1
                # If the g value of the current cell is higher then the next cell, add to heap.
                if g_val < g_vals[next_cell]:
                    parents[next_cell] = current_cell
                    g_vals[next_cell] = g_val
                    f_val = g_val + h_cost(next_cell)
                    heapq.heappush(open_cells, (f_val, next_cell))
        return None
    astar()
    
a_star("maze-Easy.txt")
a_star("maze-Medium.txt")
a_star("maze-Large.txt")
a_star("maze-VLarge.txt")

Total running time: 0.00016999244689941406
Number of nodes explored: 56
Number of steps in the path: 27
check file to find the path taken for maze-Easy.txt 

Total running time: 0.006385087966918945
Number of nodes explored: 2059
Number of steps in the path: 321
check file to find the path taken for maze-Medium.txt 

Total running time: 0.12390398979187012
Number of nodes explored: 42006
Number of steps in the path: 974
check file to find the path taken for maze-Large.txt 

Total running time: 0.972876787185669
Number of nodes explored: 274136
Number of steps in the path: 3691
check file to find the path taken for maze-VLarge.txt 



### Improvements over Depth-First Search
A star algorithm has a faster execution time for the easy and medium maze over DFS is due to the algorithm completing in a better time and space complexity. If you look at path taken for each maze that has been solved you can see that it is always the shortest path possible. Also if you examine the difference between the amount of nodes explored from A star to depth first, A star has a much smaller amount of nodes explored. As the manhattan distance is a good heuristic function for the algorithm, the time to complete is very fast. Also because the maze is quite dense in the Very Large maze, the algorithm is much faster then depth first search.

However if an A star algorithm was created with a bad heuristic function or there are multiple ways of solving the maze, depth first search may be more efficient.

## Further implementation of another algorithm
### A different algorithm for this problem would be Breadth-First Search 
BFS is a searching algorithm that goes across nodes rather than down a branch BFS. Some big advantages to BFS is that the algorithm has completeness and is optimal meaning that it always finds a solution (no loops) and that the solution is always the fastest one. However a big downside to this algorithm is the time and space complexity being high, this is because BFS may need to visit all nodes and it needs to remember all paths.

In [3]:
# time required to calculate the total time
import time

def breadth_first(maze):
    '''
    This function takes bundles everything together so that when this
    is called everything is ran at the correct time.
    
        Parameters:
            m: The file of the maze that is going to be used in the algorithm
    '''
    start_j_counter = 0
    end_i_counter = 0
    end_j_counter = 0
    nodes_visited = 0

    # Opens the file and formats contents into list of lists
    with open("mazes/"+maze, 'r') as file:
        maze = [list(line.replace(' ','').strip()) for line in file]

    # Find start and end i and j values
    start_i = 0

    for i in maze[0]:
        if i == '-':
            start_j = start_j_counter
            break
        else:
            start_j_counter += 1
            continue

    for i in maze:
        end_i_counter += 1
    end_i = end_i_counter-1

    for i in maze[-1]:
        if i == '-':
            end_j = end_j_counter
            break
        else:
            end_j_counter += 1
            continue

    # Looking at neighbouring cells from the current cell and adding 1 to their cell value
    def step(count):
        '''
        This step function looks at the each element within the maze and
        if there is a cell that is a zero next to the current cell it will
        populate the value of the next cell by the current cell plus one.
        
            parameters:
                count: the count of the current cell
        '''
        for i in range(len(maze_numbers)):
            for j in range(len(maze_numbers[i])):
                # Changes the count of the neighbouring cell to the current + 1
                if maze_numbers[i][j] == count:
                    if i > 0 and maze_numbers[i - 1][j] == 0 and maze[i - 1][j] == '-':
                        maze_numbers[i - 1][j] = count + 1
                    if j > 0 and maze_numbers[i][j - 1] == 0 and maze[i][j - 1] == '-':
                        maze_numbers[i][j - 1] = count + 1
                    if i < len(maze_numbers) - 1 and maze_numbers[i + 1][j] == 0 and maze[i + 1][j] == '-':
                        maze_numbers[i + 1][j] = count + 1
                    if j < len(maze_numbers[i]) - 1 and maze_numbers[i][j + 1] == 0 and maze[i][j + 1] == '-':
                        maze_numbers[i][j + 1] = count + 1
    
    # Create empty list of cells to be populated by the path taken
    maze_numbers = []
    # Add 0's to the maze
    for i in range(len(maze)):
        maze_numbers.append([])
        for j in range(len(maze[i])):
            maze_numbers[-1].append(0)
    # Start of with the maze entry point being 1
    maze_numbers[start_i][start_j] = 1

    start_time = time.time()
    
    count = 0
    
    # BFS taking place
    while maze_numbers[end_i][end_j] == 0:
        count += 1
        step(count)

    end_time = time.time()

    # Tracing back to find the shortest path
    n = 0
    count = maze_numbers[end_i][end_j]
    path_stack = [(end_i, end_j)]
    # Incrementing the number of nodes in the path and adding to the path stack
    # When the count hits zero it is back at the start of the maze
    while count > 1:
        if end_i > 0 and maze_numbers[end_i - 1][end_j] == count - 1:
            end_i, end_j = end_i - 1, end_j
            path_stack.append((end_i, end_j))
            n+=1
            count -= 1
        elif end_j > 0 and maze_numbers[end_i][end_j - 1] == count - 1:
            end_i, end_j = end_i, end_j - 1
            path_stack.append((end_i, end_j))
            n+=1
            count -= 1
        elif i < len(maze_numbers) - 1 and maze_numbers[end_i + 1][end_j] == count - 1:
            end_i, end_j = end_i + 1, end_j
            path_stack.append((end_i, end_j))
            n+=1
            count -= 1
        elif j < len(maze_numbers[end_i]) - 1 and maze_numbers[end_i][end_j + 1] == count - 1:
            end_i, end_j = end_i, end_j + 1
            path_stack.append((end_i, end_j))
            n+=1
            count -= 1
    # Because the path was added to the stack while backtracking it must be reversed
    path_stack.reverse()

    # For each value greater then 0 in the maze increment nodes visited by 1
    for i in range(len(maze_numbers)):
        for j in maze_numbers[i]:
            if j > 0:
                nodes_visited += 1

    print("Number of nodes in path:", n+1)
    total_time = (end_time - start_time)
    print("Time to complete bfs:", total_time)
    print("Nodes visited:", nodes_visited)
    print("Path taken:", path_stack)

    # Print the maze and the path taken
    for i in range(len(maze_numbers)):
        for j in range(len(maze_numbers[i])):
            print( str(maze_numbers[i][j]).ljust(2),end=' ')
        print()

breadth_first("maze-Easy.txt")

Number of nodes in path: 27
Time to complete bfs: 0.0004220008850097656
Nodes visited: 78
Path taken: [(0, 1), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (2, 15), (3, 15), (4, 15), (5, 15), (6, 15), (6, 16), (6, 17), (7, 17), (8, 17), (8, 18), (9, 18)]
0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
0  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 0  0  
0  0  0  0  0  7  0  0  10 0  0  13 0  0  0  17 0  19 20 0  
0  16 0  0  0  8  0  12 11 0  0  14 0  0  19 18 0  20 0  0  
0  15 0  11 10 9  0  0  0  0  0  0  0  0  0  19 0  21 22 0  
0  14 13 12 0  10 11 12 13 0  17 0  0  20 0  20 0  0  0  0  
0  15 0  0  0  0  12 0  14 15 16 17 18 19 20 21 22 23 24 0  
0  16 0  0  15 14 13 0  0  0  0  0  0  0  0  0  0  24 0  0  
0  17 18 17 16 0  14 15 0  0  0  0  0  0  0  27 26 25 26 0  
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  27 0  
